_Control

In [ ]:
%reset -f

In [1]:
#import
import os
import numpy as np
import sympy as sp
import matplotlib as mpl
import pandas as pd
import jax
import jax.numpy as jnp

dspath="./datasets"

In [3]:
def dummyf(x):
	return x**2

_Notes

In [ ]:
#matplotlib
import matplotlib.pyplot as plt
import numpy as np

"""
for i in range(decoded.shape[0]):

    array=decoded[i,0,:,:,0]
    
    # Plot the array as a grayscale image
    plt.imshow(array, cmap='gray')

    plt.colorbar()  # Add a color bar to the side for reference
    plt.title('Grayscale Image of 128x128 Array')
    plt.show()
"""

fig, axes = plt.subplots(10, 6, figsize=(12, 15))
for i in range(10):
	for j in range(6):
		array=decoded[6*i+j,0,:,:,0]
		axes[i, j].imshow(array, cmap='gray')
		axes[i, j].set_title(f"Plot {6*i+j+1}")
plt.tight_layout()
plt.show()

In [7]:
# jax checkpoints
import jax.numpy as jnp
from flax.training import checkpoints
import os
ckpt_dir = os.getcwd()+"/checkpoints/TEST"

restored={}
state = {'a':jnp.ones(16)}
#restored = checkpoints.restore_checkpoint(ckpt_dir, state, prefix=f"_ARRAY_")

checkpoints.save_checkpoint(ckpt_dir, state, 1, prefix=f"_ARRAY_", keep=5, overwrite=True)

'/home/mike_1102/python/checkpoints/TEST/_ARRAY_1'

In [5]:
def compact(func):
	func.compact=True
	return func

a0=compact(dummyf)
print(a0)

<function dummyf at 0x7f3f57b43f60>


In [ ]:
print(func(3))